<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span></li><li><span><a href="#Анализ-моделей" data-toc-modified-id="Анализ-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ моделей</a></span></li><li><span><a href="#Тестирование-лучшей-модели" data-toc-modified-id="Тестирование-лучшей-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование лучшей модели</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [2]:
df = pd.read_csv('/datasets/autos.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

DateCrawled, RegistrationMonth, DateCreated, NumberOfPictures, PostalCode, LastSeen - удалим эти столбцы, т.к. не содержат характеристик, описывающих автомобиль.

In [4]:
df.drop(['DateCrawled', 'RegistrationMonth', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'], axis=1, inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354369 non-null  int64 
 1   VehicleType       316879 non-null  object
 2   RegistrationYear  354369 non-null  int64 
 3   Gearbox           334536 non-null  object
 4   Power             354369 non-null  int64 
 5   Model             334664 non-null  object
 6   Kilometer         354369 non-null  int64 
 7   FuelType          321474 non-null  object
 8   Brand             354369 non-null  object
 9   Repaired          283215 non-null  object
dtypes: int64(4), object(6)
memory usage: 27.0+ MB


**VehicleType**

In [6]:
df['VehicleType'].unique()

array([nan, 'coupe', 'suv', 'small', 'sedan', 'convertible', 'bus',
       'wagon', 'other'], dtype=object)

Заменим пропуски в `VehicleType` на 'other'

In [7]:
df['VehicleType'] = df['VehicleType'].fillna('other')

In [8]:
df['VehicleType'].unique()

array(['other', 'coupe', 'suv', 'small', 'sedan', 'convertible', 'bus',
       'wagon'], dtype=object)

**RegistrationYear**

In [9]:
df['RegistrationYear'].sort_values().unique()

array([1000, 1001, 1039, 1111, 1200, 1234, 1253, 1255, 1300, 1400, 1500,
       1600, 1602, 1688, 1800, 1910, 1915, 1919, 1920, 1923, 1925, 1927,
       1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938,
       1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950,
       1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961,
       1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972,
       1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983,
       1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994,
       1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2066, 2200, 2222, 2290, 2500, 2800, 2900, 3000,
       3200, 3500, 3700, 3800, 4000, 4100, 4500, 4800, 5000, 5300, 5555,
       5600, 5900, 5911, 6000, 6500, 7000, 7100, 7500, 7800, 8000, 8200,
       8455, 8500, 8888, 9000, 9229, 9450, 9996, 99

In [10]:
df[(df['RegistrationYear'] <= 1969) | (df['RegistrationYear'] > 2019)]['RegistrationYear'].count()/len(df)

0.0040720266163236625

Оставим данные о машинах, зарегистрированных  в период с 1970 по 2019,  т.к. это практически 99% датасета

In [11]:
df = df[(df['RegistrationYear'] > 1969) | (df['RegistrationYear'] <= 2019)]

**Gearbox**

In [12]:
df['Gearbox'].value_counts(dropna=False)

manual    268251
auto       66285
NaN        19833
Name: Gearbox, dtype: int64

Заменим NaN на категорию 'NAN'

In [13]:
df['Gearbox'] = df['Gearbox'].fillna('NAN')

**Model**

In [14]:
df['Model'].sort_values().unique()

array(['100', '145', '147', '156', '159', '1_reihe', '1er', '200',
       '2_reihe', '300c', '3_reihe', '3er', '4_reihe', '500', '5_reihe',
       '5er', '601', '6_reihe', '6er', '7er', '80', '850', '90', '900',
       '9000', '911', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a8',
       'a_klasse', 'accord', 'agila', 'alhambra', 'almera', 'altea',
       'amarok', 'antara', 'arosa', 'astra', 'auris', 'avensis', 'aveo',
       'aygo', 'b_klasse', 'b_max', 'beetle', 'berlingo', 'bora',
       'boxster', 'bravo', 'c1', 'c2', 'c3', 'c4', 'c5', 'c_klasse',
       'c_max', 'c_reihe', 'caddy', 'calibra', 'captiva', 'carisma',
       'carnival', 'cayenne', 'cc', 'ceed', 'charade', 'cherokee',
       'citigo', 'civic', 'cl', 'clio', 'clk', 'clubman', 'colt', 'combo',
       'cooper', 'cordoba', 'corolla', 'corsa', 'cr_reihe', 'croma',
       'crossfire', 'cuore', 'cx_reihe', 'defender', 'delta', 'discovery',
       'doblo', 'ducato', 'duster', 'e_klasse', 'elefantino', 'eos',
       'escort', 'espac

In [15]:
df['Model'].isna().sum()

19705

Заменим пропуски на 'other'

In [16]:
df['Model'] = df['Model'].fillna('other')

In [17]:
df['Model'].sort_values().unique()

array(['100', '145', '147', '156', '159', '1_reihe', '1er', '200',
       '2_reihe', '300c', '3_reihe', '3er', '4_reihe', '500', '5_reihe',
       '5er', '601', '6_reihe', '6er', '7er', '80', '850', '90', '900',
       '9000', '911', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a8',
       'a_klasse', 'accord', 'agila', 'alhambra', 'almera', 'altea',
       'amarok', 'antara', 'arosa', 'astra', 'auris', 'avensis', 'aveo',
       'aygo', 'b_klasse', 'b_max', 'beetle', 'berlingo', 'bora',
       'boxster', 'bravo', 'c1', 'c2', 'c3', 'c4', 'c5', 'c_klasse',
       'c_max', 'c_reihe', 'caddy', 'calibra', 'captiva', 'carisma',
       'carnival', 'cayenne', 'cc', 'ceed', 'charade', 'cherokee',
       'citigo', 'civic', 'cl', 'clio', 'clk', 'clubman', 'colt', 'combo',
       'cooper', 'cordoba', 'corolla', 'corsa', 'cr_reihe', 'croma',
       'crossfire', 'cuore', 'cx_reihe', 'defender', 'delta', 'discovery',
       'doblo', 'ducato', 'duster', 'e_klasse', 'elefantino', 'eos',
       'escort', 'espac

**Kilometer**

In [18]:
df['Kilometer'].describe()

count    354369.000000
mean     128211.172535
std       37905.341530
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: Kilometer, dtype: float64

Проблем не обнаружено

**FuelType**

In [19]:
df['FuelType'].unique()

array(['petrol', 'gasoline', nan, 'lpg', 'other', 'hybrid', 'cng',
       'electric'], dtype=object)

Заменим пропуски на 'other'

In [20]:
df['FuelType'] = df['FuelType'].fillna('other')

**Brand**

In [21]:
df['Brand'].unique()

array(['volkswagen', 'audi', 'jeep', 'skoda', 'bmw', 'peugeot', 'ford',
       'mazda', 'nissan', 'renault', 'mercedes_benz', 'opel', 'seat',
       'citroen', 'honda', 'fiat', 'mini', 'smart', 'hyundai',
       'sonstige_autos', 'alfa_romeo', 'subaru', 'volvo', 'mitsubishi',
       'kia', 'suzuki', 'lancia', 'toyota', 'chevrolet', 'dacia',
       'daihatsu', 'trabant', 'saab', 'chrysler', 'jaguar', 'daewoo',
       'porsche', 'rover', 'land_rover', 'lada'], dtype=object)

Исправления не требуются

**Repaired**

In [22]:
df['Repaired'].unique()

array([nan, 'yes', 'no'], dtype=object)

Заменим пропуски на 'NAN'

In [23]:
df['Repaired'] = df['Repaired'].fillna('NAN')

**Power**

In [24]:
df['Power'].sort_values().unique()

array([    0,     1,     2,     3,     4,     5,     6,     7,     8,
           9,    10,    11,    12,    13,    14,    15,    16,    17,
          18,    19,    20,    21,    22,    23,    24,    25,    26,
          27,    28,    29,    30,    31,    32,    33,    34,    35,
          36,    37,    38,    39,    40,    41,    42,    43,    44,
          45,    46,    47,    48,    49,    50,    51,    52,    53,
          54,    55,    56,    57,    58,    59,    60,    61,    62,
          63,    64,    65,    66,    67,    68,    69,    70,    71,
          72,    73,    74,    75,    76,    77,    78,    79,    80,
          81,    82,    83,    84,    85,    86,    87,    88,    89,
          90,    91,    92,    93,    94,    95,    96,    97,    98,
          99,   100,   101,   102,   103,   104,   105,   106,   107,
         108,   109,   110,   111,   112,   113,   114,   115,   116,
         117,   118,   119,   120,   121,   122,   123,   124,   125,
         126,   127,

Заменим значения мощности двигателя меньшие 50 и большие 500 на медианные значения характерные для этих моделей

In [25]:
for col in df['Model'].unique():
    df.loc[(df['Model'] == col) & ((df['Power'] <=50) | (df['Power'] >=500)), 'Power'] = \
    df.loc[(df['Model'] == col), 'Power'].median()


In [26]:
df['Power'].sort_values().unique()

array([  0. ,  26. ,  34. ,  42. ,  50. ,  51. ,  52. ,  53. ,  54. ,
        55. ,  56. ,  57. ,  58. ,  59. ,  60. ,  61. ,  62. ,  63. ,
        64. ,  65. ,  66. ,  67. ,  68. ,  68.5,  69. ,  70. ,  71. ,
        72. ,  73. ,  74. ,  75. ,  76. ,  77. ,  78. ,  79. ,  80. ,
        81. ,  82. ,  83. ,  84. ,  85. ,  86. ,  87. ,  88. ,  89. ,
        90. ,  91. ,  92. ,  93. ,  94. ,  95. ,  96. ,  97. ,  97.5,
        98. ,  99. , 100. , 101. , 102. , 103. , 104. , 105. , 106. ,
       107. , 108. , 109. , 110. , 111. , 112. , 113. , 114. , 115. ,
       115.5, 116. , 117. , 118. , 119. , 120. , 121. , 122. , 123. ,
       124. , 125. , 126. , 127. , 128. , 129. , 130. , 131. , 132. ,
       133. , 134. , 135. , 136. , 137. , 138. , 139. , 140. , 141. ,
       142. , 143. , 144. , 145. , 146. , 147. , 148. , 149. , 150. ,
       151. , 152. , 153. , 154. , 155. , 156. , 157. , 158. , 159. ,
       160. , 161. , 162. , 163. , 164. , 165. , 166. , 167. , 168. ,
       169. , 170. ,

Для тех моделей, у которых мощность двигателя меньше 50, заменим значение на максимальное значание, характерное для это модели.

In [27]:
for col in df['Model'].unique():
    df.loc[(df['Model'] == col) & ((df['Power'] <=50)), 'Power'] = \
    df.loc[(df['Model'] == col), 'Power'].max()


In [28]:
df[df['Power'] == 0]

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,Repaired
234296,3800,wagon,1978,manual,0.0,serie_1,30000,gasoline,land_rover,NAN
280216,0,other,1970,NAN,0.0,serie_1,100000,petrol,land_rover,NAN


Эти записи удалим

In [29]:
df = df[df['Power'] > 0]

**Price**

In [30]:
df[df['Price'] <= 50].count()

Price               12749
VehicleType         12749
RegistrationYear    12749
Gearbox             12749
Power               12749
Model               12749
Kilometer           12749
FuelType            12749
Brand               12749
Repaired            12749
dtype: int64

Удалим записи со стоимостью автомобиля меньше 50

In [31]:
df = df[df['Power'] >= 50]

**Дубликаты**

In [32]:
df.duplicated().sum()

47577

In [33]:
df = df.drop_duplicates().reset_index(drop=True)

**Кодирование категориальных признаков**

```python
# порядковое кодирование
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=9999)
cat_columns = [список всех категориальных признаков]
encoder.fit(features_train[cat_columns])

features_train_ord = features_train.copy()
features_valid_ord = features_valid.copy()
features_test_ord = features_test.copy()

features_train_ord[cat_columns] = encoder.transform(features_train_ord[cat_columns])
features_valid_ord[cat_columns] = encoder.transform(features_valid_ord[cat_columns])
features_test_ord[cat_columns] = encoder.transform(features_test_ord[cat_columns])
```

**Вывод:**

1. Были обработаны пропуски, в основном пропуски не удалялись, а заменялись на NAN или other
2. Не типчные значения мощностей были заменены на медианные значения, характерные для этих моделей
3. Удалено 12 тыс. записей с ценой меньше 50
4. Удалено 47 тыс. дубликатов
5. Осуществлено кодирование категориальных признаков методом OHE

## Обучение моделей

In [35]:
features = df.drop(['Price'], axis=1)
target = df['Price']

In [36]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)
features_train, features_val, target_train, target_val = train_test_split(features_train, target_train, test_size=0.25, random_state=12345)

**Кодирование категориальных признаков**

In [37]:
cat_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']

OneHotEncoder - для градиентного бустинга

In [38]:
ohe_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_encoder.fit(features_train[cat_columns])

features_train_ohe = ohe_encoder.transform(features_train[cat_columns])
features_train_ohe = pd.DataFrame(features_train_ohe, columns=ohe_encoder.get_feature_names(cat_columns))
features_train_ohe.index = features_train.index
features_train_ohe = pd.concat([features_train.drop(cat_columns, axis=1), features_train_ohe], axis=1)

features_val_ohe = ohe_encoder.transform(features_val[cat_columns])
features_val_ohe = pd.DataFrame(features_val_ohe, columns=ohe_encoder.get_feature_names(cat_columns))
features_val_ohe.index = features_val.index
features_val_ohe = pd.concat([features_val.drop(cat_columns, axis=1), features_val_ohe], axis=1)

features_test_ohe = ohe_encoder.transform(features_test[cat_columns])
features_test_ohe = pd.DataFrame(features_test_ohe, columns=ohe_encoder.get_feature_names(cat_columns))
features_test_ohe.index = features_test.index
features_test_ohe = pd.concat([features_test.drop(cat_columns, axis=1), features_test_ohe], axis=1)

OrdinalEncoder - для деревьев

In [39]:
oe_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=9999)
cat_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']
oe_encoder.fit(features_train[cat_columns])

features_train_oe = features_train.copy()
features_val_oe = features_val.copy()
features_test_oe = features_test.copy()

features_train_oe[cat_columns] = oe_encoder.transform(features_train_oe[cat_columns])
features_val_oe[cat_columns] = oe_encoder.transform(features_val_oe[cat_columns])
features_test_oe[cat_columns] = oe_encoder.transform(features_test_oe[cat_columns])

**LightGBM**

In [40]:
data = lgb.Dataset(features_train_ohe, label=target_train)

Осуществим подбор с помощью GridsearchCV

In [41]:
model = lgb.LGBMRegressor()

In [42]:
param_grid = {
    'num_leaves': [20, 40],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100],
    'reg_alpha': [0.0, 0.1],
    'random_state': [12345]
}

In [43]:
grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', verbose=2)

In [44]:
grid_search.fit(features_train_ohe, target_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END learning_rate=0.01, n_estimators=100, num_leaves=20, random_state=12345, reg_alpha=0.0; total time= 4.1min
[CV] END learning_rate=0.01, n_estimators=100, num_leaves=20, random_state=12345, reg_alpha=0.0; total time= 2.2min
[CV] END learning_rate=0.01, n_estimators=100, num_leaves=20, random_state=12345, reg_alpha=0.0; total time=  40.7s
[CV] END learning_rate=0.01, n_estimators=100, num_leaves=20, random_state=12345, reg_alpha=0.0; total time=   6.8s
[CV] END learning_rate=0.01, n_estimators=100, num_leaves=20, random_state=12345, reg_alpha=0.0; total time=   6.2s
[CV] END learning_rate=0.01, n_estimators=100, num_leaves=20, random_state=12345, reg_alpha=0.1; total time=   7.3s
[CV] END learning_rate=0.01, n_estimators=100, num_leaves=20, random_state=12345, reg_alpha=0.1; total time=  51.8s
[CV] END learning_rate=0.01, n_estimators=100, num_leaves=20, random_state=12345, reg_alpha=0.1; total time= 2.1min
[CV] END lea

GridSearchCV(estimator=LGBMRegressor(),
             param_grid={'learning_rate': [0.01, 0.1], 'n_estimators': [100],
                         'num_leaves': [20, 40], 'random_state': [12345],
                         'reg_alpha': [0.0, 0.1]},
             scoring='neg_mean_squared_error', verbose=2)

In [45]:
best_params_lgbm = grid_search.best_params_
print("Наилучшие параметры  для LGM:", best_params_lgbm)

Наилучшие параметры  для LGM: {'learning_rate': 0.1, 'n_estimators': 100, 'num_leaves': 40, 'random_state': 12345, 'reg_alpha': 0.0}


In [46]:
model_1 = lgb.LGBMRegressor(learning_rate=0.1, n_estimators=100, num_leaves=40, reg_alpha=0.0, random_state=12345)

Замерим время обучения модели с лучшеми параметрами.

In [47]:
start_time = time.time()
model_1.fit(features_train_ohe, target_train)
end_time = time.time()
time_train_1 = end_time-start_time

Замерим время предсказания модели на валидационной выборке

In [49]:
start_time = time.time()
y_pred_1 = model_1.predict(features_val_ohe)
end_time = time.time()
time_prediction_1 = end_time-start_time

Вычеслим RMSE на валидационной выборке

In [51]:
# Вычисление метрики RMSE
rmse_valid_1 = mean_squared_error(target_val, y_pred_1, squared=False)

**DecisionTreeRegressor**

In [59]:
model = DecisionTreeRegressor()

In [60]:
param_dist = {
    'max_depth': [None, 5, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'random_state': [12345]
}

In [61]:
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=50, scoring='neg_mean_squared_error', random_state=12345)

In [62]:
random_search.fit(features_train_oe, target_train)

RandomizedSearchCV(estimator=DecisionTreeRegressor(), n_iter=50,
                   param_distributions={'max_depth': [None, 5, 10, 20, 30],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'random_state': [12345]},
                   random_state=12345, scoring='neg_mean_squared_error')

In [63]:
best_params_dtr = random_search.best_params_
print("Наилучшие параметры:", best_params_dtr)

Наилучшие параметры: {'random_state': 12345, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 20}


In [64]:
model_2 = DecisionTreeRegressor(random_state=12345, min_samples_split=10, min_samples_leaf=4, max_features='sqrt', max_depth=20)

In [65]:
start_time = time.time()
model_2.fit(features_train_oe, target_train)
end_time = time.time()
time_train_2 = end_time-start_time

In [66]:
start_time = time.time()
#Делаем предсказание на валидационной выборке
y_pred_2 = model_2.predict(features_val_oe)
end_time = time.time()
time_prediction_2 = end_time-start_time

In [67]:
# Вычисляем RMSE
rmse_valid_2 = mean_squared_error(target_val, y_pred_2, squared=False)

**Вывод**
 
 1. Было обучено 2 модели
     - LightGBM, параметры подбирались с помощью GridSearch
     - DecisionTreeRegressor, параметры подбирались с помощью RandomSearch
 2. На валидационной выборке расчитаны метрики RMSE
 3. Измерены время обучения и время предсказания моделей

## Анализ моделей

Сравним результаты, полученые на предыдущем шаге.

In [68]:
index = ['LightGBM', 'DTR']
columns = ['Время обучения', 'Время предсказания', 'RMSE-val']
data = np.array([
        [time_train_1, time_train_2],
        [time_prediction_1, time_prediction_2],
        [rmse_valid_1, rmse_valid_2]])

result = pd.DataFrame(data=data.T, columns=columns, index=index)
result

,Время обучения,Время предсказания,RMSE-val
LightGBM,6.772949,2.329466,1845.362749
DTR,0.206056,0.023076,2090.722556


**Вывод**

Обе модели проходят по критерию оценки, но т.к. для заказчика также важны скорость обучения и предсказания, то для тестирования отбираем модель DTR

## Тестирование лучшей модели

Получим RMSE на тестовых данных.

In [69]:
y_pred_2 = model_2.predict(features_test_oe)
rmse_test_2 = mean_squared_error(target_test, y_pred_2, squared=False)
print('RMSE DTR  на тестовых данных', rmse_test_2)

RMSE DTR  на тестовых данных 2101.2095932121388


**Вывод:**
- модель  DTR на тестовых данных удовлетворяет требованиям к оценке, при этом показывает более высокую скорость по сравнению с моделью LGBM. 